# SD3 Dreambooth / LoRA training

boilerplate for using diffusers in cloud envs

In [ ]:
# delete diffusers repo from env if necessary
!rm -rf diffusers

## Clone the diffusers repository

In [ ]:
!git clone https://github.com/huggingface/diffusers
!cd diffusers && pip install -e .
!pip install wandb bitsandbytes huggingface_hub sentencepiece "peft>=0.6.0"

In [ ]:
!cd diffusers/examples/dreambooth && pip install -r requirements_sd3.txt
!accelerate config default

## Download Dataset (optional)

In [ ]:
# Not required if using your own dataset...
# If using data from hub, change names
# If local data, upload and skip
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

## HuggingFace Login

In [ ]:
from huggingface_hub import login, logout

login("[YOUR_HUGGINGFACE_API_KEY_HERE]") # CHANGE ME

# Required to access model (if pulling)

## Set Paths

In [ ]:
# Set environment variables
import os

os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-3-medium-diffusers"
os.environ["INSTANCE_DIR"] = "dog" # dataset dir
print("done")

In [ ]:
# remove .huggingface in dataset dir if causes error
!rm -rf dog/.huggingface

## Run LoRA Training

In [ ]:
# LoRA (dreambooth)
os.environ["OUTPUT_DIR"] = "trained-sd3-lora" # name output if needed

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

!accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sd3.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=25 \
  --seed="0" \
  --use_8bit_adam
  #--push_to_hub # remove comment to push model to huggingface account

## Run Dreambooth Training

In [ ]:
# Dreambooth

export OUTPUT_DIR="trained-sd3-dreambooth" # name output if needed

!accelerate launch train_dreambooth_sd3.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="A photo of sks dog in a bucket" \
  --validation_epochs=25 \
  --seed="0" \
  #--push_to_hub # remove comment to push model to huggingface account